In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
independent=dataset[["Age","EstimatedSalary","Gender_Male"]]
dependent=dataset["Purchased"]

In [9]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,True
1,35,20000,True
2,26,43000,False
3,27,57000,False
4,19,76000,True
...,...,...,...
395,46,41000,False
396,51,23000,True
397,50,20000,False
398,36,33000,True


In [10]:
dependent

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [14]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [15]:
from sklearn.naive_bayes import CategoricalNB

from sklearn.model_selection import GridSearchCV

param_grid = {
     'alpha': [0.1, 0.5, 1.0],
    'fit_prior': [True, False],
    'min_categories': [None, 3, 5]
    
    }

grid = GridSearchCV(CategoricalNB(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# Fit the model
grid.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(estimator=CategoricalNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0], 'fit_prior': [True, False],
                         'min_categories': [None, 3, 5]},
             scoring='f1_weighted', verbose=3)

In [16]:
re=grid.cv_results_

grid_predictions = grid.predict(X_test) 
   
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [17]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'alpha': 0.1, 'fit_prior': True, 'min_categories': None}: 0.8251506483229226


In [18]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[77  8]
 [15 34]]


In [19]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.84      0.91      0.87        85
           1       0.81      0.69      0.75        49

    accuracy                           0.83       134
   macro avg       0.82      0.80      0.81       134
weighted avg       0.83      0.83      0.83       134



In [20]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.870828331332533

In [21]:
table=pd.DataFrame.from_dict(re)

In [22]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_fit_prior,param_min_categories,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.050391,0.004945,0.013171,0.000574,0.1,True,None,"{'alpha': 0.1, 'fit_prior': True, 'min_categor...",0.735616,0.743112,0.710831,0.805463,NaN,NaN,NaN,1
1,0.048510,0.004843,0.012922,0.001136,0.1,True,3,"{'alpha': 0.1, 'fit_prior': True, 'min_categor...",0.735616,0.743112,0.710831,0.805463,NaN,NaN,NaN,1
2,0.039156,0.000399,0.012298,0.001041,0.1,True,5,"{'alpha': 0.1, 'fit_prior': True, 'min_categor...",0.735616,0.743112,0.710831,0.805463,NaN,NaN,NaN,1
3,0.050315,0.011812,0.014135,0.002638,0.1,False,None,"{'alpha': 0.1, 'fit_prior': False, 'min_catego...",0.719468,0.748608,0.732285,0.849057,NaN,NaN,NaN,1
4,0.041848,0.005884,0.012091,0.001373,0.1,False,3,"{'alpha': 0.1, 'fit_prior': False, 'min_catego...",0.719468,0.748608,0.732285,0.849057,NaN,NaN,NaN,1
5,0.043692,0.007137,0.009796,0.001711,0.1,False,5,"{'alpha': 0.1, 'fit_prior': False, 'min_catego...",0.719468,0.748608,0.732285,0.849057,NaN,NaN,NaN,1
6,0.044171,0.006541,0.012442,0.000794,0.5,True,None,"{'alpha': 0.5, 'fit_prior': True, 'min_categor...",0.751984,0.782633,0.688741,0.783837,NaN,NaN,NaN,1
7,0.040571,0.002410,0.012143,0.001198,0.5,True,3,"{'alpha': 0.5, 'fit_prior': True, 'min_categor...",0.751984,0.782633,0.688741,0.783837,NaN,NaN,NaN,1
8,0.040947,0.001861,0.011995,0.002815,0.5,True,5,"{'alpha': 0.5, 'fit_prior': True, 'min_categor...",0.751984,0.776838,0.688741,0.783837,NaN,NaN,NaN,1
9,0.039959,0.003897,0.013039,0.002978,0.5,False,None,"{'alpha': 0.5, 'fit_prior': False, 'min_catego...",0.719468,0.811321,0.732285,0.832483,NaN,NaN,NaN,1


In [24]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age:43
Salary:50000
Sex Male 0 or 1:0


In [25]:
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]]) 
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


C:\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
